In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import matplotlib.pyplot as plt

import sys
sys.path.append('../../module/')

import keras2
from keras2.models import Model

from keras2.layers import concatenate, Dense, Input, Flatten, merge
from keras2.optimizers import Adam
from util import moving_average, dlqr, discretized_system, lqr
import gym2
from rl2.agents import selfDDPGAgent, selfDDPGAgent2
from rl2.memory import SequentialMemory

Using TensorFlow backend.
Using TensorFlow backend.


In [13]:
def critic_net(a_shape , s_shape):
    action_input = Input(a_shape)
    observation_input = Input(shape=(1,)+s_shape)
    flattened_observation = Flatten()(observation_input)
    x = concatenate([action_input, flattened_observation])
    x = Dense(16, activation="relu")(x)
    x = Dense(16, activation="relu")(x)
    x = Dense(1, activation="linear")(x)
    critic = Model(inputs=[action_input, observation_input], outputs=x)
    critic.summary()
    return (critic, action_input)

In [14]:
critic = critic_net((2,),(2,))[0]
critic.compile(loss='mean_squared_error',optimizer='adam')
critic.load_weights('./saved_agent/2000000_critic.h5')

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            (None, 1, 2)         0                                            
__________________________________________________________________________________________________
input_5 (InputLayer)            (None, 2)            0                                            
__________________________________________________________________________________________________
flatten_3 (Flatten)             (None, 2)            0           input_6[0][0]                    
__________________________________________________________________________________________________
concatenate_3 (Concatenate)     (None, 4)            0           input_5[0][0]                    
                                                                 flatten_3[0][0]                  
__________

In [17]:
batch = []
x_high = np.array([np.pi, 8])
for _ in range(1000):
    x = np.random.uniform(low=-high, high=high)
    a = np.random.uniform(low=np.array([-10.,0.001]), high=np.array([10.,0.1]))
    batch.append([a,np.array([x])])
batch = np.array(batch)
print(batch)

[[array([2.6204674 , 0.04301175]) array([[-2.3130588 , -3.37846919]])]
 [array([9.84650368, 0.01832604]) array([[-2.16050941,  1.08178002]])]
 [array([-9.39918975,  0.08716014]) array([[-0.11833389, -2.29205352]])]
 ...
 [array([0.62305494, 0.00394506]) array([[-0.53681996, -0.91611221]])]
 [array([3.61111459, 0.05204357]) array([[ 1.26111678, -1.39256534]])]
 [array([6.71405334, 0.03976292]) array([[-2.05277237, -0.21928759]])]]


In [18]:
critic.predict_on_batch(batch)

ValueError: Error when checking model input: the list of Numpy arrays that you are passing to your model is not the size the model expected. Expected to see 2 array(s), but instead got the following list of 1 arrays: [array([[array([2.6204674 , 0.04301175]),
        array([[-2.3130588 , -3.37846919]])],
       [array([9.84650368, 0.01832604]),
        array([[-2.16050941,  1.08178002]])],
       [array([-9.3991897...